# Attentional RNN for Sentiment Analysis
### Then we highlight the key information with attentional mask

Get the dataset for [IMDB comment](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz)

In [1]:
SEQ_LEN = 300
HIDDEN_SIZE = 50
VOCAB_SIZE = 5000

In [2]:
import os
from glob import glob
import pandas as pd
import numpy as np

In [3]:
train_pos = glob("/data/aclImdb/train/pos/*")
train_neg = glob("/data/aclImdb/train/neg/*")

test_pos = glob("/data/aclImdb/test/pos/*")
test_neg = glob("/data/aclImdb/test/neg/*")

In [4]:
print(len(train_pos))
print(len(train_neg))

print(len(test_pos))
print(len(test_neg))

12500
12500
12500
12500


In [5]:
def read_file(path):
    """ a function to read the file path and return the text string contained"""
    f = open(path,errors="ignore")
    txt = f.read()
    f.close()
    return txt

In [6]:
train_df_1 = pd.DataFrame({"path":train_pos})
train_df_2 = pd.DataFrame({"path":train_neg})
train_df_1["label"] = 1
train_df_2["label"] = 0
train_df = pd.concat([train_df_1,train_df_2])

In [7]:
train_df["text"] = train_df.path.apply(read_file)

In [8]:
train_df.sample(20)

,path,label,text
1597,/data/aclImdb/train/neg/11000_1.txt,0,Words cannot describe how utterly abysmal this...
4302,/data/aclImdb/train/neg/12141_3.txt,0,"...because 99 out of 100 times, the producers ..."
2134,/data/aclImdb/train/pos/4838_10.txt,1,This movie reminded me that some old Black & W...
12063,/data/aclImdb/train/pos/9420_8.txt,1,The Man in the White Suit is one of those deli...
902,/data/aclImdb/train/neg/1183_2.txt,0,"This, and Immoral Tales, both left a bad taste..."
9545,/data/aclImdb/train/pos/9615_9.txt,1,This is a great movie! Most of us have seen Ju...
6913,/data/aclImdb/train/neg/2830_3.txt,0,If Ashanti had been a serious attempt at a fil...
3584,/data/aclImdb/train/pos/6621_8.txt,1,I just came back from a pre-release viewing of...
4181,/data/aclImdb/train/pos/3364_10.txt,1,I too was quite astonished to see how few peop...
11603,/data/aclImdb/train/neg/9076_1.txt,0,"This was surely the stupidest, crudest, most r..."


In [9]:
test_df_1 = pd.DataFrame({"path":test_pos})
test_df_2 = pd.DataFrame({"path":test_neg})
test_df_1["label"] = 1
test_df_2["label"] = 0
test_df = pd.concat([test_df_1,test_df_2])

In [10]:
test_df["text"] = test_df.path.apply(read_file)

In [11]:
test_df.sample(20)

,path,label,text
9854,/data/aclImdb/test/neg/6803_1.txt,0,What were they thinking when they made this tr...
9851,/data/aclImdb/test/pos/7891_10.txt,1,I finally have seen the real reason why Peter ...
6859,/data/aclImdb/test/pos/7050_8.txt,1,Luckily for Bill Murray this is such a light-w...
3880,/data/aclImdb/test/pos/5596_9.txt,1,Race Against Fear has to be one of the most mo...
3579,/data/aclImdb/test/pos/3701_10.txt,1,It's 3:30am.<br /><br />I just saw this movie ...
10171,/data/aclImdb/test/pos/5278_8.txt,1,"Rendered in beautiful water colors, Ponyo At T..."
299,/data/aclImdb/test/neg/7919_2.txt,0,I think that this is a disappointing sequel. I...
599,/data/aclImdb/test/neg/3327_1.txt,0,"My daughter liked it but I was aghast, that a ..."
2423,/data/aclImdb/test/neg/3375_3.txt,0,"Yes, in this movie you are treated to multiple..."
3630,/data/aclImdb/test/pos/7100_7.txt,1,"This film marked the end of the ""serious"" Univ..."


Shuffle the data

In [12]:
train_df = train_df.sample(frac = 1.).reset_index()
test_df = test_df.sample(frac = 1.).reset_index()

### Data Batchly Feeder

In [13]:
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader

In [14]:
CUDA = torch.cuda.is_available()
CUDA

True

In [15]:
from keras.preprocessing.sequence import pad_sequences

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [16]:
from collections import Counter
class seq_data(Dataset):
    def __init__(self,df, vocab_path, seq_addr, vocab_size, build_vocab = False,):
        """
        vocab_in,vocab_out are csv file addresses
        """
        self.df = df
        self.vocab_path = vocab_path
        self.seq_addr = seq_addr
        self.vocab_size = vocab_size
        
        print("[Loading the sequence data]")
        
        self.seq = list(df["text"])
        
        self.N = len(self.seq)
        print("Length of sequence:\t",self.N)
        
        if build_vocab:
            self.vocab = self.build_vocab(self.seq)
            self.vocab.to_csv(self.vocab_path, index = False)
        else:
            self.vocab = pd.read_csv(self.vocab_path).fillna("")
            
        self.print_vocab_info()
        
        print("building mapping dicts")
        self.char2idx,self.idx2char = self.get_mapping(self.vocab)
        
    def __len__(self):
        return self.N
    
    def __getitem__(self,idx):
        return self.seq2idx(self.seq[idx],self.mapfunc),idx
    
    def get_full_token(self,list_of_tokens):
        """
        From a list of list of tokens, to a long list of tokens, duplicate tokens included
        """
        return (" ".join(list_of_tokens)).split(" ")
    
    def get_mapping(self,vocab_df):
        char2idx=dict(zip(vocab_df["token"],vocab_df["idx"]))
        idx2char=dict(zip(vocab_df["idx"],vocab_df["token"]))
        return char2idx,idx2char
    
    def seq2idx(self,x,mapfunc):
        return np.vectorize(mapfunc)(x.split(" ")).tolist()
    
    def mapfunc(self,x):
        try:
            return self.char2idx[x]
        except:
            return 1
        
    def get_token_count_dict(self,full_token):
        """count the token to a list"""
        return Counter(full_token)
    
    def build_vocab(self,seq_list):
        ct_dict = self.get_token_count_dict(self.get_full_token(seq_list))
        ct_dict["SOS_TOKEN"] = 9e9
        ct_dict["EOS_TOKEN"] = 8e9
        ct_dict[" "] = 7e9
        tk,ct = list(ct_dict.keys()),list(ct_dict.values())
        
        token_df=pd.DataFrame({"token":tk,"count":ct}).sort_values(by="count",ascending=False)
        return token_df.reset_index().drop("index",axis=1).reset_index().rename(columns={"index":"idx"}).fillna("")[:self.vocab_size]
    
    def print_vocab_info(self):
        self.vocab_size = len(self.vocab)
        
        print("[seq vocab]: %s,\t%s total lines"%(self.vocab,self.vocab_size))
            
        print("Sequence vocab samples:")
        print(self.vocab.sample(5))

# We have to self difine a collate function
# becuz we take the longest sequence lengnth with in a batch as the seq length for the entire batch
def pad_collate(batch):
    nlp,label_idx = zip(*batch)
    nlp_arr = pad_sequences(nlp, maxlen = SEQ_LEN,padding="post",)
    
    return torch.LongTensor(nlp_arr),torch.FloatTensor(label_arr[np.array(label_idx)])

def pad_collate_valid(batch):
    nlp,label_idx = zip(*batch)
    nlp_arr = pad_sequences(nlp, maxlen = SEQ_LEN,padding="post",)
    
    return torch.LongTensor(nlp_arr),torch.FloatTensor(label_arr_valid[np.array(label_idx)])

### Prepare the dataset, setting up the mapping dictionary,  save the vocabulary...

In [17]:
train_ds = seq_data(train_df,
              "/data/review_vocab.csv",
              "/data/review_seq_0.0.1.npy",
              5000,
              build_vocab=True)

[Loading the sequence data]
Length of sequence:	 25000
[seq vocab]:        idx         count        token
0        0  9.000000e+09    SOS_TOKEN
1        1  8.000000e+09    EOS_TOKEN
2        2  7.000000e+09             
3        3  2.870290e+05          the
4        4  1.550960e+05            a
5        5  1.526530e+05          and
6        6  1.429710e+05           of
7        7  1.325680e+05           to
8        8  1.032280e+05           is
9        9  8.557600e+04           in
10      10  6.597200e+04            I
11      11  6.455800e+04         that
12      12  5.719800e+04         this
13      13  5.443200e+04           it
14      14  5.093500e+04        /><br
15      15  4.669700e+04          was
16      16  4.251000e+04           as
17      17  4.172000e+04         with
18      18  4.106800e+04          for
19      19  3.378400e+04          but
20      20  3.376200e+04          The
21      21  3.076600e+04           on
22      22  3.050400e+04        movie
23      23  2.849800

In [18]:

valid_ds = seq_data(test_df,
              "/data/review_vocab.csv",
              "/data/review_seq_0.0.2.npy",
              5000,
              build_vocab=False)

[Loading the sequence data]
Length of sequence:	 25000
[seq vocab]:        idx         count        token
0        0  9.000000e+09    SOS_TOKEN
1        1  8.000000e+09    EOS_TOKEN
2        2  7.000000e+09             
3        3  2.870290e+05          the
4        4  1.550960e+05            a
5        5  1.526530e+05          and
6        6  1.429710e+05           of
7        7  1.325680e+05           to
8        8  1.032280e+05           is
9        9  8.557600e+04           in
10      10  6.597200e+04            I
11      11  6.455800e+04         that
12      12  5.719800e+04         this
13      13  5.443200e+04           it
14      14  5.093500e+04        /><br
15      15  4.669700e+04          was
16      16  4.251000e+04           as
17      17  4.172000e+04         with
18      18  4.106800e+04          for
19      19  3.378400e+04          but
20      20  3.376200e+04          The
21      21  3.076600e+04           on
22      22  3.050400e+04        movie
23      23  2.849800

In [19]:
label_arr = train_df["label"].as_matrix().reshape(-1,1)
label_arr_valid = test_df["label"].as_matrix().reshape(-1,1)

In [20]:
dl = DataLoader(train_ds, batch_size=4, shuffle = True,collate_fn=pad_collate)
gen = iter(dl)
test_nlp, test_y = next(gen)

In [21]:
test_nlp.size(), test_y.size()

(torch.Size([4, 300]), torch.Size([4, 1]))

## Model

In [22]:
from torch.nn import functional as F

In [23]:
class attn_lstm(nn.Module):
    def __init__(self,seq_len,vocab_size,hidden_size,num_layers = 1):
        """
        seq_len: sequence length
        """
        super(attn_lstm,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.emb = nn.Embedding(vocab_size,hidden_size,)
        self.lstm = nn.LSTM(hidden_size,hidden_size,batch_first = True,num_layers = num_layers)
        self.attn = nn.Linear(hidden_size,1,bias = False)
        
    def forward(self,x_input):
        embedded = self.emb(x_input)
        attn_mask = self.attn(embedded)
        output,(h,c) = self.lstm(embedded)
        output = output.permute(0,2,1)
        output_hidden = output.bmm(attn_mask).squeeze(-1)
        return output_hidden, attn_mask
        

In [24]:
class attn_net(nn.Module):
    def __init__(self, seq_len = SEQ_LEN, hidden_size = HIDDEN_SIZE, vocab_size = VOCAB_SIZE):
        super(attn_net,self).__init__()
        self.attn_lstm = attn_lstm(seq_len, vocab_size, hidden_size)
        self.mlp = nn.Sequential(*[
            nn.Dropout(.2),
            nn.Linear(hidden_size, 256,bias = False),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256,1,bias = False)
        ])
    
    def forward(self, x):
        output, attn_mask = self.attn_lstm(x)
        return self.mlp(output),attn_mask
        

In [25]:
def accuracy(y_pred, y_true):
    return (y_pred>.5).long().eq(y_true.long()).float().mean()

In [26]:
def action(*args, **kwargs):
    x, y = args[0]
    if CUDA:
        x,y = x.cuda(),y.cuda()
    opt.zero_grad()
    y_,attn_mask = attn_model(x)
    loss = loss_func(y_,y)
    acc = accuracy(y_,y)
    loss.backward()
    
    opt.step()
    if kwargs["ite"] == 10:
        torch.save(attn_model.state_dict(),"/data/review_nlp_analysis_0.0.1.npy")
    return {"loss":loss.item(),"acc":acc.item()}

def val_action(*args, **kwargs):
    x, y = args[0]
    if CUDA:
        x,y = x.cuda(),y.cuda()
    y_,attn_mask = attn_model(x)
    loss = loss_func(y_,y)
    acc = accuracy(y_,y)
    return {"loss":loss.item(),"acc":acc.item()}

In [27]:
attn_model = attn_net()
# attn_model.load_state_dict(torch.load("/data/review_nlp_analysis_0.0.2.npy"))
if CUDA:
    torch.cuda.empty_cache()
    attn_model.cuda()
from torch.optim import Adam

opt = Adam(attn_model.parameters())
loss_func = nn.BCEWithLogitsLoss()
# loss_func = nn.MSELoss()

In [28]:
from ray.matchbox import Trainer
trainer = Trainer(train_ds,val_dataset=valid_ds,batch_size=16,print_on=2)
trainer.train_data.collate_fn = pad_collate
trainer.val_data.collate_fn = pad_collate_valid
trainer.action = action
trainer.val_action = val_action
trainer.train(1)

😎[val_ep_0_i_1562]	acc	0.805😂	loss	0.403: 100%|██████████| 1563/1563 [00:24<00:00, 63.10it/s]


In [29]:
trainer.train(1)

😎[val_ep_0_i_1562]	acc	0.838😂	loss	0.354: 100%|██████████| 1563/1563 [00:24<00:00, 62.94it/s]


## Visualize Using The Attentional Mask

In [30]:
from IPython.core.display import display, HTML

In [31]:
dl = DataLoader(train_ds,batch_size = 256, shuffle=False, collate_fn=pad_collate)
gen = iter(dl)
seq = train_ds.seq[:256]
x,y = next(gen)
if CUDA:
    x,y = x.cuda(),y.cuda()

In [32]:
x.size()

torch.Size([256, 300])

In [33]:
y_, mask = attn_model(x)

In [34]:
if CUDA:
    y_,mask = y.cpu(),mask.cpu()

In [35]:
mask

tensor([[[-2.8670e-01],
         [ 5.2415e-02],
         [ 2.7802e-01],
         ...,
         [-6.5229e-03],
         [-6.5229e-03],
         [-6.5229e-03]],

        [[ 2.9653e-02],
         [ 6.0128e-01],
         [ 2.7703e-01],
         ...,
         [ 2.9653e-02],
         [-2.9299e-01],
         [ 2.9653e-02]],

        [[ 2.9653e-02],
         [ 2.9653e-02],
         [ 8.9614e-01],
         ...,
         [-6.5229e-03],
         [-6.5229e-03],
         [-6.5229e-03]],

        ...,

        [[ 2.4202e-02],
         [-6.2384e-02],
         [-1.9808e-01],
         ...,
         [-1.2462e+00],
         [-1.9824e-01],
         [ 2.9653e-02]],

        [[-3.7494e-01],
         [ 8.2393e-01],
         [ 8.2571e-01],
         ...,
         [-6.5229e-03],
         [-6.5229e-03],
         [-6.5229e-03]],

        [[-2.8670e-01],
         [ 1.8085e-01],
         [ 1.6467e-01],
         ...,
         [-6.5229e-03],
         [-6.5229e-03],
         [-6.5229e-03]]])

In [36]:
def paint_comment(seq_line, mask,y,y_):
    seq_line = seq_line.split(" ")
    seq_line = seq_line[:20]
    mask = mask[:20]
    if len(seq_line) > 10:
        mask_p = F.softmax(torch.clamp(mask[:len(seq_line)],0,10).squeeze(),dim=0).data.numpy()
        mask_n = F.softmax((torch.clamp(mask[:len(seq_line)],-10,0)*(-1)).squeeze(),dim=0).data.numpy()
        p = "".join(list("<span style = 'color:rgba(253,10,253,%s)'>_%s</span>"%(hl*0.8+0.2, word) for word,hl in zip(seq_line, mask_p)))
        n = "".join(list("<span style = 'color:rgba(253,10,10,%s)'>_%s</span>"%(hl*0.8+0.2, word) for word,hl in zip(seq_line, mask_n)))
        print("label:%s\t guess:%s"%(y.item(),y_.item()))
        display(HTML(p))
        display(HTML(n))

In [37]:
for i in range(256):
    paint_comment(seq[i],mask[i],y[i],y_[i])

label:1.0	 guess:1.0


label:0.0	 guess:0.0


label:1.0	 guess:1.0


label:0.0	 guess:0.0


label:0.0	 guess:0.0


...

label:1.0	 guess:1.0


label:0.0	 guess:0.0


label:1.0	 guess:1.0


label:0.0	 guess:0.0


label:0.0	 guess:0.0
